# Demo: ML vs Rule-Based Predictions (user1)

This notebook initializes the app data, signs in as `user1`, shows ML-powered predictions, demonstrates rule-based fallback, and resets demo data for a clean next run.

In [13]:
# Setup: add repo to path, initialize databases, ensure user1 data
import sys, sqlite3
from pathlib import Path
repo = Path().resolve().parent
sys.path.insert(0, str(repo))

# Initialize auth DB + ensure tables
from data.auth.init_auth_db import init_auth_db
init_auth_db()

# Ensure session_audit exists (in addition to run_app initialization)
db_path = repo / 'data' / 'user_data.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS session_audit (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  user_id INTEGER NOT NULL,
  compound TEXT NOT NULL,
  weight REAL,
  reps INTEGER,
  rpe REAL,
  deviation_reason TEXT,
  prediction_source TEXT,
  recommended_weight REAL,
  actual_weight REAL,
  accuracy_delta REAL,
  prediction_status TEXT DEFAULT 'pending',
  logged_at TIMESTAMP
)
""")
conn.commit(); conn.close()

# Enable ML for user1 across compounds and populate history
from enable_ml_user1 import enable_ml_for_user1
enable_ml_for_user1()
from populate_user1_history import populate_user1_history
populate_user1_history()

print('✓ Setup complete: auth DB, model flags, and user1 history ready.')

[OK] Database already initialized (ensured CSVs exist)
ENABLING ML FOR USER1
✓ Found user1 with ID: 11
✓ model_quality table ready

  Setting squat to ML-enabled...
    ✓ squat enabled with session_count=14, model_mape=0.08, model_enabled=1

  Setting bench_press to ML-enabled...
    ✓ bench_press enabled with session_count=14, model_mape=0.08, model_enabled=1

  Setting lat_pulldown to ML-enabled...
    ✓ lat_pulldown enabled with session_count=14, model_mape=0.08, model_enabled=1

  Setting seated_row to ML-enabled...
    ✓ seated_row enabled with session_count=14, model_mape=0.08, model_enabled=1

✓ ML ENABLED FOR USER1

user1 will now use ML predictions instead of rule-based!
POPULATING USER1 WORKOUT HISTORY

1. Loading baseline data from: C:\Users\rezas\GitHub\delta-sequential-recommender\data\baseline\strong_4krows_baseline_data.csv
   ✓ Loaded 3989 rows
   ✓ Filtered to 1295 top sets
   ✓ Mapped to compounds, 94 rows with valid exercises
   ℹ  Compound distribution:
      squat:

## Sign In
We sign into the application as `user1` and fetch the user data path for predictions.

In [14]:
# Sign in as user1
from src import auth
user_id, user_data_path = auth.login('user1', 'demo123')
print({'user_id': user_id, 'user_data_path': user_data_path})

{'user_id': 11, 'user_data_path': 'C:\\Users\\rezas\\GitHub\\delta-sequential-recommender\\users\\user1'}


## Demo: ML Predictions
We fetch the last workout for each compound and ask the recommendation engine for the next weight.
You'll see `Source: model` when ML is used, and `Source: rule_based` otherwise.

In [15]:
import pandas as pd
from pathlib import Path

# Reload modules to get latest code
import importlib
import sys
if 'src.recommendation_engine' in sys.modules:
    del sys.modules['src.recommendation_engine']

from src.recommendation_engine import get_recommendation, ModelCache

print("=" * 70)
print("ML-POWERED RECOMMENDATIONS")
print("=" * 70)

compounds = ['squat', 'bench_press', 'lat_pulldown']
for c in compounds:
    csv = Path(user_data_path) / f'user1_{c}_history.csv'
    hist = pd.read_csv(csv)
    last = hist.iloc[-1]
    
    rec, source, reason = get_recommendation(
        user_id=user_id,
        user_data_path=str(user_data_path),
        compound=c,
        last_weight=last['weight'],
        last_reps=int(last['reps']),
        last_rpe=last.get('rpe', 8.0),
        session_count=len(hist)
    )
    
    # Get model accuracy
    accuracy = ModelCache.get_accuracy(c)
    
    original = round(last['weight'], 1)
    recommended = round(rec, 1) if rec is not None else None
    
    # Format output: Original → Recommended (Accuracy%)
    print(f"\n{c.upper().replace('_', ' ')}")
    print(f"  Original: {original} lb")
    print(f"  → Recommended: {recommended} lb ({source})")
    print(f"  Model Accuracy: {accuracy}% | {reason}")

print("\n" + "=" * 70)

ML-POWERED RECOMMENDATIONS

SQUAT
  Original: 155.0 lb
  → Recommended: 105.9 lb (model)
  Model Accuracy: 74% | ML model (calibrated)

BENCH PRESS
  Original: 22.5 lb
  → Recommended: 27.3 lb (model)
  Model Accuracy: 50% | ML model (calibrated)

LAT PULLDOWN
  Original: 8.8 lb
  → Recommended: 25.6 lb (model)
  Model Accuracy: 64% | ML model (calibrated)



## Demo: Rule-Based Fallback
To demonstrate fallback, we temporarily disable ML for one compound (`lat_pulldown`).
With ML disabled and limited sessions, the engine falls back to rule-based.

In [16]:
# Demonstrate weak model triggering fallback
import sqlite3
import pandas as pd
from pathlib import Path
import sys

# Reload to get latest ModelCache
if 'src.recommendation_engine' in sys.modules:
    del sys.modules['src.recommendation_engine']
if 'src.rule_based' in sys.modules:
    del sys.modules['src.rule_based']

from src.recommendation_engine import get_recommendation, ModelCache

print("=" * 70)
print("FALLBACK DEMO: BENCH PRESS (Weak ML Model)")
print("=" * 70)

auth_db = repo / 'data' / 'auth' / 'app_users.db'
conn = sqlite3.connect(auth_db)
cur = conn.cursor()

# bench_press model is intentionally weak (50% accuracy)
# Let's see what it predicts vs rule-based fallback
bp_csv = Path(user_data_path) / 'user1_bench_press_history.csv'
bp_hist = pd.read_csv(bp_csv)
last = bp_hist.iloc[-1]

# With weak model but ML enabled, should still try ML
rec_ml, source_ml, reason_ml = get_recommendation(
    user_id=user_id,
    user_data_path=str(user_data_path),
    compound='bench_press',
    last_weight=last['weight'],
    last_reps=int(last['reps']),
    last_rpe=last.get('rpe', 8.0),
    session_count=len(bp_hist)
)

accuracy_bp = ModelCache.get_accuracy('bench_press')
original = round(last['weight'], 1)
recommended = round(rec_ml, 1) if rec_ml is not None else None

print(f"\nWEAK MODEL (50% accuracy):")
print(f"  Original: {original} lb")
print(f"  → ML Recommended: {recommended} lb ({source_ml})")
print(f"  Model Accuracy: {accuracy_bp}%")
print(f"  Note: {reason_ml}")

# Now disable ML for bench_press to force rule-based
cur.execute("UPDATE model_quality SET model_enabled = 0 WHERE user_id = ? AND compound = ?", (user_id, 'bench_press'))
conn.commit()

rec_rb, source_rb, reason_rb = get_recommendation(
    user_id=user_id,
    user_data_path=str(user_data_path),
    compound='bench_press',
    last_weight=last['weight'],
    last_reps=int(last['reps']),
    last_rpe=last.get('rpe', 8.0),
    session_count=len(bp_hist)
)

recommended_rb = round(rec_rb, 1) if rec_rb is not None else None
print(f"\nFORCED FALLBACK (ML Disabled):")
print(f"  Original: {original} lb")
print(f"  → Rule-Based Recommendation: {recommended_rb} lb ({source_rb})")
print(f"  Reason: {reason_rb}")

# Re-enable ML for bench_press
cur.execute("UPDATE model_quality SET model_enabled = 1 WHERE user_id = ? AND compound = ?", (user_id, 'bench_press'))
conn.commit()
conn.close()

print("\n✓ ML re-enabled for bench_press")
print("=" * 70)

FALLBACK DEMO: BENCH PRESS (Weak ML Model)

WEAK MODEL (50% accuracy):
  Original: 22.5 lb
  → ML Recommended: 27.3 lb (model)
  Model Accuracy: 50%
  Note: ML model (calibrated)

FORCED FALLBACK (ML Disabled):
  Original: 22.5 lb
  → Rule-Based Recommendation: 22.5 lb (rule_based)
  Reason: hold_near_limit

✓ ML re-enabled for bench_press


## Cleanup
Reset `user1` history to the baseline validation slice so future demos start clean. Also clear `session_audit` entries for user1 to avoid leftover pending/completed predictions.

In [17]:
# Reset user1 history and clear session audit rows
from populate_user1_history import populate_user1_history
populate_user1_history()
print('✓ user1 history reset to baseline demo slice.')

# Clear session_audit entries for user1
import sqlite3
from src.auth import get_user_id
db_path = repo / 'data' / 'user_data.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()
uid = get_user_id('user1')
cur.execute("DELETE FROM session_audit WHERE user_id = ?", (uid,))
conn.commit(); conn.close()
print('✓ Cleared session_audit entries for user1.')

POPULATING USER1 WORKOUT HISTORY

1. Loading baseline data from: C:\Users\rezas\GitHub\delta-sequential-recommender\data\baseline\strong_4krows_baseline_data.csv
   ✓ Loaded 3989 rows
   ✓ Filtered to 1295 top sets
   ✓ Mapped to compounds, 94 rows with valid exercises
   ℹ  Compound distribution:
      squat: 15 sets
      bench_press: 46 sets
      lat_pulldown: 33 sets
      seated_row: 0 sets

2. Processing squat...
   ✓ Wrote 14 sessions to user1_squat_history.csv
     Weight range: 125.0 - 175.0 lbs
     Average RPE: 8.6
     Load deltas: -50.0 to 50.0 lbs

2. Processing bench_press...
   ✓ Wrote 14 sessions to user1_bench_press_history.csv
     Weight range: 22.5 - 35.0 lbs
     Average RPE: 8.5
     Load deltas: -7.5 to 7.5 lbs

2. Processing lat_pulldown...
   ✓ Wrote 14 sessions to user1_lat_pulldown_history.csv
     Weight range: 8.8 - 8.8 lbs
     Load deltas: 0.0 to 0.0 lbs

2. Processing seated_row...
   ⚠  Only 0 rows for seated_row. Using all available.
   ⚠  No data fo